In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import glob  # 파일 경로를 쉽게 다루기 위한 라이브러리
import os    # 운영체제와 상호작용하기 위한 라이브러리

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
base_directory = '/content/drive/MyDrive/ddareung/2023'

# 모든 하위 폴더를 포함하여 CSV 파일 찾기
file_pattern = os.path.join(base_directory, '**/*.csv')
file_paths = glob.glob(file_pattern, recursive=True)

if not file_paths:
    print("--- 오류 발생 ---")
    print(f"경로 '{base_directory}' 및 그 하위 폴더에서 CSV 파일을 찾지 못했습니다.")
else:
    print(f"--- 파일 찾기 성공 ---")
    print(f"총 {len(file_paths)}개의 CSV 파일을 찾았습니다.")

    all_dfs = []
    # for 반복문 내부가 수정되었습니다.
    for path in file_paths:
        try:
            # 1순위: 가장 표준적인 'utf-8'로 파일을 읽어봅니다.
            df = pd.read_csv(path, encoding='utf-8')
            all_dfs.append(df)
        except UnicodeDecodeError:
            try:
                # 2순위: 'utf-8' 실패 시 'cp949'로 다시 읽어봅니다.
                df = pd.read_csv(path, encoding='cp949')
                all_dfs.append(df)
            except Exception as e:
                # 두 가지 인코딩 모두 실패하는 경우, 어떤 파일인지 알려주고 넘어갑니다.
                print(f"!!! 경고: '{os.path.basename(path)}' 파일은 utf-8과 cp949로 읽을 수 없습니다. 원인: {e}")
        except Exception as e:
             print(f"!!! 경고: '{os.path.basename(path)}' 파일 처리 중 예상치 못한 오류 발생. 원인: {e}")


    # 모든 데이터프레임을 성공적으로 읽어왔는지 확인
if not all_dfs:
    print("\n읽어온 데이터가 없습니다. 파일 처리 중 문제가 발생했을 수 있습니다.")
else:
    try:
        # 리스트에 담긴 모든 데이터프레임을 하나로 합칩니다.
        combined_df = pd.concat(all_dfs, ignore_index=True)
        print(f"\n모든 파일의 데이터를 성공적으로 합쳤습니다. (총 {len(combined_df)} 행)")

        # 데이터 처리
        combined_df.rename(columns={'대여시간': '시간대'}, inplace=True)

        # 최종 누적 결과 출력
        print("\n--- [전체 누적] 연령대별 이용 건수 ---")
        total_age_counts = combined_df['연령대코드'].value_counts()
        print(total_age_counts)

        print("\n--- [전체 누적] 시간대별 이용 건수 ---")
        total_time_counts = combined_df['시간대'].value_counts().sort_index()
        print(total_time_counts)


        print("\n--- [전체 누적] 대여소별 이용 건수  ---")
        # '대여소번호'별로 이용 건수를 세고, 가장 많이 이용된 순서대로 정렬합니다.
        # .head(20)을 사용하여 상위 20개만 출력합니다. (숫자는 원하시는 만큼 조절 가능)
        total_station_counts = combined_df['대여소번호'].value_counts()
        print(total_station_counts)

    except Exception as e:
        print(f"\n데이터 병합 및 분석 중 오류가 발생했습니다: {e}")


--- 파일 찾기 성공 ---
총 11개의 CSV 파일을 찾았습니다.

모든 파일의 데이터를 성공적으로 합쳤습니다. (총 38038756 행)

--- [전체 누적] 연령대별 이용 건수 ---
연령대코드
20대      12207003
30대       9272841
40대       5723365
50대       3971001
기타        3184599
~10대      2192682
60대       1295486
70대이상      191779
Name: count, dtype: int64

--- [전체 누적] 시간대별 이용 건수 ---
시간대
0      860570
1      591175
2      389080
3      266829
4      218543
5      393190
6      819639
7     1825360
8     2573237
9     1642389
10    1333089
11    1523352
12    1683695
13    1690565
14    1725208
15    1926364
16    2341128
17    2976708
18    3513881
19    2605470
20    2238144
21    2036202
22    1722456
23    1142482
Name: count, dtype: int64

--- [전체 누적] 대여소별 이용 건수  ---
대여소번호
2715    99973
4217    85310
502     82390
2102    76572
1153    75377
        ...  
4343      100
4845       35
11         17
4339       17
9979        3
Name: count, Length: 2784, dtype: int64
